In [1]:
# Libraries and functions
import sys
sys.path.append("../functions/")
from model import compute_ifr_bygender
from matrices import create_matrices_dict
from calibration import calibrate_model, run_posterior_sampling

import numpy as np
import pandas as pd
import json
from datetime import datetime, timedelta
import os.path

# global variables
n_comp = 6                                                                                 # number of compartments
age_groups = ['20-29', '30-39', '40-49', '50-59', '60+']                                   # list of strings with age groups 
age_groups_bins = [20, 30, 40, 50, 60, np.inf]                                             # list of int with age groups extremes 
n_age = len(age_groups)                                                                    # number of age groups
n_gen = 2                                                                                  # number of gender groups
model_list = ['0', 'I', 'B', 'IB', 'C', 'CI', 'CB', 'CIB']                                 # list of strings with model codes 
n_sim = 1000000                                                                            # number of simulations to perform
save_interval = 1000                                                                       # number of simulations after which a saving is performed

path_save = '../runs/'                                                                     # path where to save simulations

In [2]:
# DATA IMPORT

# Load the weekly mortality data
df_deaths = pd.read_csv('../data/deaths/weekly_deaths.csv')

# Load the population array (Nij)
with open('../data/population/Nij.json', 'r') as f:
    Nij = np.array(json.load(f))

# For contact matrices

# Load the CoMix contact matrices by zones stratified by both age and gender
with open('../data/matrices/CM_zones_dict.json', 'r') as f:
    # Load and immediately convert lists back to numpy arrays
    CM_zones_dict = {k: np.array(v) for k, v in json.load(f).items()}

# Load the CoMix contact matrices by zones stratified only by age
with open('../data/matrices/CM_zones_nogender_dict.json', 'r') as f:
    # Load and immediately convert lists back to numpy arrays
    CM_zones_nogender_dict = {k: np.array(v) for k, v in json.load(f).items()}

# Load the POLYMOD contact matrices by zones stratified by both age and gender
with open('../data/matrices/CM_polymod_dict.json', 'r') as f:
    # Load and immediately convert lists back to numpy arrays
    CM_polymod_dict = {k: np.array(v) for k, v in json.load(f).items()}

# Load the POLYMOD contact matrices by zones stratified only by age
with open('../data/matrices/CM_polymod_nogender_dict.json', 'r') as f:
    # Load and immediately convert lists back to numpy arrays
    CM_polymod_nogender_dict = {k: np.array(v) for k, v in json.load(f).items()}

# Load the pre-processed mobility reduction data to update Polymod matrices
df_mob = pd.read_csv('../data/mobility/mobility_changes.csv', index_col='week')

# Load data of restrictions by region to update CoMix matrices
data_zones_byregion = pd.read_csv('../data/restrictions/zones_byregion.csv', index_col='Region')

# Load data of population by region to update CoMix matrices
data_population_byregion = pd.read_csv('../data/population/population_byregion.csv', index_col='Region')

In [3]:
# Parameters

t_step = 1                                                          # temporal step (in days)
initial_date = datetime(2020, 9, 14)                                # starting date of the simulation
end_date = datetime(2021, 2, 21)                                    # ending date of the simulation
t_max = int((end_date - initial_date).days * (1/t_step))            # number of days in the simulation

mu = 1 / 2.5                                                        # inverse of infectious period
epsilon = 1 / 4                                                     # inverse of latent period
Delta = 14                                                          # Average number of days of delay of deaths reporting

IFR = [0.000309, 0.000844, 0.00161, 0.00595, 0.0328]                # Infectious Fatality Ratio by age group
OR_gender = 1.39
IFR_gender = compute_ifr_bygender(IFR, OR_gender, Nij)             # Infectious Fatality Ratio by age group and gender

initial_cases = 10119                                               # Observed initial number of individuals in L and I compartments (14th Septemeber)

threshold = 0.2                                                     # Threshold of calibration metric for acceptance

# Define priors
prior_dict = {
        'i0':  lambda: np.random.randint(int(initial_cases), int(10 * initial_cases)) / Nij.sum(),             # Initial fraction of individuals in L and I compartments
        'r0':  lambda: np.random.uniform(0.03, 0.10),                                                          # Initial fraction of recovered individuals
        'R01': lambda: np.random.uniform(1.0, 1.7),                                                            # R0 at the beginning of the simulations
        'R02': lambda: np.random.uniform(0.5, 1.7),                                                            # R0 on November 6th, 2020
        'r_beta': lambda: np.random.uniform(1.0, 1.3)                                                          # Relative susceptibility for males (used if 'B' in model_type).
    }

# Compute period (initial and final week) on which performing the calibration 
# initial_week: 2 weeks after initial_date to give the model time to adjust
initial_week = (initial_date + timedelta(days=14)).strftime("%Y-%W")
initial_week = f"{int(initial_week[:4]) - 1}-52" if initial_week.endswith('-00') else initial_week    # adjust for week 00 if necessary

# final_week: week of end_date
final_week = end_date.strftime("%Y-%W")
final_week = f"{int(final_week[:4]) - 1}-52" if final_week.endswith('-00') else final_week            # adjust for week 00 if necessary

# Define observed weekly deaths for calibration
df_deaths = df_deaths.groupby('week')['weekly_deaths'].sum().reset_index()
deaths_observed = df_deaths.loc[(df_deaths['week'] >= initial_week) & (df_deaths['week'] <= final_week)]
deaths_observed = deaths_observed['weekly_deaths'].values

# Create the dictionary of contact matrices 
CM_dates_gender = create_matrices_dict(initial_date, t_max, t_step, df_mob, data_zones_byregion, data_population_byregion, CM_polymod_dict, CM_zones_dict)
CM_dates_nogender = create_matrices_dict(initial_date, t_max, t_step, df_mob, data_zones_byregion, data_population_byregion, CM_polymod_nogender_dict, CM_zones_nogender_dict)

In [4]:
simulations_dict = {}

# iterate through each model version (e.g., '0', 'I', 'B', 'IB', etc.)
for model_string in model_list:
    
    # define the output file path for saving raw calibration results
    output_file = path_save + f'raw/simulations_model_{model_string}.csv'

    # select the appropriate Contact Matrices based on model type
    # if 'C' is present, use gender-stratified matrices (10x10); otherwise, use age-only (5x5)
    if 'C' in model_string:
        CM_dates_dict = CM_dates_gender
    else:
        CM_dates_dict = CM_dates_nogender

    # select the appropriate IFR vector/matrix based on model type
    # if 'I' is present, use gender-stratified IFR; otherwise, use age-only vector
    if 'I' in model_string:
        IFR_model = IFR_gender
    else:
        IFR_model = IFR

    # check if simulation results already exist to avoid re-running expensive calibration
    if os.path.exists(output_file):
        simulations_dict[model_string] = pd.read_csv(output_file)
    else:
        # run calibration: generates n_sim simulations and saves accepted parameters/outcomes to CSV
        simulations_dict[model_string] = calibrate_model(model_string, Nij, CM_dates_dict, initial_date, t_max, t_step, initial_cases, mu, epsilon, IFR_model, Delta, deaths_observed, threshold, n_sim, save_interval, 
                                                         prior_dict, output_file)
    
    # parse the 'simulated_deaths' column from JSON strings back into Python lists
    simulations_dict[model_string]['simulated_deaths'] = simulations_dict[model_string]['simulated_deaths'].apply(json.loads)

In [5]:
# Select best simulations
best_simulations_dict = {}
for model_string, df_simulations in simulations_dict.items():
    # Filter for error below threshold
    df_best_simulations = df_simulations[df_simulations['err']<=threshold].copy().reset_index(drop=True)
    best_simulations_dict[model_string] = df_best_simulations

In [6]:
# Configuration parameters
n_param_sets = 1000
n_sim_per_set = 10

# iterate through each model version (e.g., '0', 'I', 'B', 'IB', etc.)
for model_string, df_best_simulations in best_simulations_dict.items():

    # select the appropriate Contact Matrices based on model type
    # if 'C' is present, use gender-stratified matrices (10x10); otherwise, use age-only (5x5)
    if 'C' in model_string:
        CM_dates_dict = CM_dates_gender
    else:
        CM_dates_dict = CM_dates_nogender

    # select the appropriate IFR vector/matrix based on model type
    # if 'I' is present, use gender-stratified IFR; otherwise, use age-only vector
    if 'I' in model_string:
        IFR_model = IFR_gender
    else:
        IFR_model = IFR
        
    # define final path
    file_path = path_save + f"sampled/sampled_simulations_model_{model_string}.pkl.gz"
    
    # check if path already exists 
    if os.path.exists(file_path):
        print(f"File already exists for model {model_string}, skipping...")
    else:
        # run posterior sampling function
        df_sampled = run_posterior_sampling(model_string, Nij, CM_dates_dict, initial_date, t_max, t_step, mu, epsilon, IFR_model, Delta, df_best_simulations, n_param_sets, n_sim_per_set,
                                            file_path)

File already exists for model 0, skipping...
File already exists for model I, skipping...
File already exists for model B, skipping...
File already exists for model IB, skipping...
File already exists for model C, skipping...
File already exists for model CI, skipping...
File already exists for model CB, skipping...
File already exists for model CIB, skipping...
